In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile10 = pd.read_pickle("FFF_Train_Decile10.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile10.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile10, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 10')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 38/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2493 - acc: 0.9097 - val_loss: 0.2237 - val_acc: 0.9058
Epoch 39/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2420 - acc: 0.9136 - val_loss: 0.2233 - val_acc: 0.9045
Epoch 40/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2451 - acc: 0.9080 - val_loss: 0.2228 - val_acc: 0.9058
Epoch 41/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2463 - acc: 0.9113 - val_loss: 0.2212 - val_acc: 0.9058
Epoch 42/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2446 - acc: 0.9087 - val_loss: 0.2201 - val_acc: 0.9058
Epoch 43/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2432 - acc: 0.9087 - val_loss: 0.2184 - val_acc: 0.9071
Epoch 44/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2432 - acc: 0.9087 - val_loss: 0.2174 - val_acc: 0.9084
Epoch 45/100
3055/3055 [=================

Epoch 55/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.1920 - acc: 0.9237 - val_loss: 0.1694 - val_acc: 0.9332
Epoch 56/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.1896 - acc: 0.9237 - val_loss: 0.1695 - val_acc: 0.9332
Epoch 57/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1884 - acc: 0.9254 - val_loss: 0.1694 - val_acc: 0.9332
Epoch 58/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1882 - acc: 0.9257 - val_loss: 0.1694 - val_acc: 0.9332
Epoch 59/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1889 - acc: 0.9264 - val_loss: 0.1693 - val_acc: 0.9332
Epoch 60/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1943 - acc: 0.9254 - val_loss: 0.1694 - val_acc: 0.9332
Epoch 61/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.1885 - acc: 0.9257 - val_loss: 0.1693 - val_acc: 0.9332
Epoch 62/150
3055/3055 [=================

3055/3055 [==============================] - 12s 4ms/step - loss: 0.2112 - acc: 0.9188 - val_loss: 0.1942 - val_acc: 0.9188
Epoch 24/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2130 - acc: 0.9198 - val_loss: 0.1919 - val_acc: 0.9293
Epoch 25/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2185 - acc: 0.9201 - val_loss: 0.1885 - val_acc: 0.9215
Epoch 26/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2229 - acc: 0.9188 - val_loss: 0.1975 - val_acc: 0.9293
Epoch 27/150
3055/3055 [==============================] - 11s 4ms/step - loss: 0.2159 - acc: 0.9162 - val_loss: 0.1931 - val_acc: 0.9188
Epoch 28/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2113 - acc: 0.9191 - val_loss: 0.1885 - val_acc: 0.9280
Epoch 29/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2131 - acc: 0.9205 - val_loss: 0.1878 - val_acc: 0.9228
Epoch 30/150
3055/3055 [==============================

Epoch 82/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2014 - acc: 0.9218 - val_loss: 0.1833 - val_acc: 0.9332
Epoch 83/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2034 - acc: 0.9198 - val_loss: 0.1839 - val_acc: 0.9332
Epoch 84/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2051 - acc: 0.9211 - val_loss: 0.1834 - val_acc: 0.9332
Epoch 85/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2017 - acc: 0.9237 - val_loss: 0.1837 - val_acc: 0.9332
Epoch 86/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2002 - acc: 0.9201 - val_loss: 0.1831 - val_acc: 0.9332
Epoch 87/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2082 - acc: 0.9188 - val_loss: 0.1833 - val_acc: 0.9332
Epoch 88/150
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2066 - acc: 0.9188 - val_loss: 0.1833 - val_acc: 0.9332
Epoch 89/150
3055/3055 [=================

...

3055/3055 [==============================] - 12s 4ms/step - loss: 0.1889 - acc: 0.9300 - val_loss: 0.1638 - val_acc: 0.9398
Epoch 140/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.1880 - acc: 0.9293 - val_loss: 0.1637 - val_acc: 0.9398
Epoch 141/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.1882 - acc: 0.9270 - val_loss: 0.1641 - val_acc: 0.9385
Epoch 142/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1874 - acc: 0.9286 - val_loss: 0.1641 - val_acc: 0.9398
Epoch 143/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.1876 - acc: 0.9300 - val_loss: 0.1642 - val_acc: 0.9398
Epoch 144/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1898 - acc: 0.9267 - val_loss: 0.1646 - val_acc: 0.9411
Epoch 145/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.1915 - acc: 0.9264 - val_loss: 0.1644 - val_acc: 0.9411
Epoch 146/200
3055/3055 [=======================

3055/3055 [==============================] - 13s 4ms/step - loss: 0.2318 - acc: 0.9149 - val_loss: 0.2164 - val_acc: 0.9162
Epoch 58/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2370 - acc: 0.9129 - val_loss: 0.2157 - val_acc: 0.9175
Epoch 59/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2347 - acc: 0.9149 - val_loss: 0.2160 - val_acc: 0.9149
Epoch 60/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2336 - acc: 0.9152 - val_loss: 0.2142 - val_acc: 0.9162
Epoch 61/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2363 - acc: 0.9162 - val_loss: 0.2138 - val_acc: 0.9162
Epoch 62/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2360 - acc: 0.9191 - val_loss: 0.2120 - val_acc: 0.9149
Epoch 63/100
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2345 - acc: 0.9165 - val_loss: 0.2108 - val_acc: 0.9188
Epoch 64/100
3055/3055 [==============================

Epoch 16/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2103 - acc: 0.9241 - val_loss: 0.1866 - val_acc: 0.9280
Epoch 17/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2100 - acc: 0.9198 - val_loss: 0.1888 - val_acc: 0.9293
Epoch 18/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2143 - acc: 0.9227 - val_loss: 0.1841 - val_acc: 0.9319
Epoch 19/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2056 - acc: 0.9260 - val_loss: 0.1802 - val_acc: 0.9359
Epoch 20/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2088 - acc: 0.9214 - val_loss: 0.1810 - val_acc: 0.9306
Epoch 21/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2096 - acc: 0.9198 - val_loss: 0.1799 - val_acc: 0.9359
Epoch 22/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.2033 - acc: 0.9241 - val_loss: 0.1869 - val_acc: 0.9280
Epoch 23/200
3055/3055 [=================

3055/3055 [==============================] - 12s 4ms/step - loss: 0.1901 - acc: 0.9290 - val_loss: 0.1683 - val_acc: 0.9385
Epoch 134/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.1913 - acc: 0.9247 - val_loss: 0.1683 - val_acc: 0.9385
Epoch 135/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1907 - acc: 0.9257 - val_loss: 0.1683 - val_acc: 0.9385
Epoch 136/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.1935 - acc: 0.9244 - val_loss: 0.1683 - val_acc: 0.9385
Epoch 137/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1891 - acc: 0.9264 - val_loss: 0.1683 - val_acc: 0.9385
Epoch 138/200
3055/3055 [==============================] - 13s 4ms/step - loss: 0.1909 - acc: 0.9270 - val_loss: 0.1682 - val_acc: 0.9385
Epoch 139/200
3055/3055 [==============================] - 12s 4ms/step - loss: 0.1912 - acc: 0.9280 - val_loss: 0.1682 - val_acc: 0.9385
Epoch 140/200
3055/3055 [=======================

1504/1504 [==============================] - 2s 1ms/step
[0.15291024665249156, 0.9375]
Best performing model chosen hyper-parameters:
{'epochs': 3, 'lr': 0}
Confusion Matrix for LSTM predictions:
[[ 332   55]
 [  39 1078]]
Classification Report for LSTM predictions:
-------------------------------------------
             precision    recall  f1-score   support

          0       0.89      0.86      0.88       387
          1       0.95      0.97      0.96      1117

avg / total       0.94      0.94      0.94      1504

Accuracy score: 0.9375


In [3]:
0.93484

0.93484

In [4]:
0.930

0.93

In [ ]:
#0.94